In [39]:
import pickle
from config import model_folder
from config import reference_model_folder

def load_subword_slot(model_name):
    with open(model_name, 'rb') as f:
        params = pickle.load(f)
        subword_slot = params['subword_slot']
        subword2index = params['subword2index']
        del params
    return subword_slot, subword2index

def print_subword_df_distribution(subword_slot, subword2index, words):
    for word in words:
        idx = subword2index.get(word, -1)
        if idx == -1: continue
        print('\n{}'.format(word), (100*subword_slot[idx,:]).max())
        print(100*subword_slot[idx,:])

model_name = '{}/subword_df_slot.pkl'.format(model_folder)
subword_slot, subword2index = load_subword_slot(model_name)
index2subword = [word for word, _ in sorted(subword2index.items(),key=lambda x:x[1])]

In [52]:
print_subword_df_distribution(subword_slot, subword2index, '입니다 팝니다 겨울 오빠 하던데 극장 엔진오일 상영 엔진 차량 악장 휴양지 터미네이터 구매 리스 영화 로봇 커피 미국 선수'.split())


입니다 50.281
[ 32.65625   36.15625   25.90625   33.28125   35.15625   35.5625     0.
  43.1875    41.90625    0.        31.171875  41.34375   43.75      45.46875
  41.375     42.34375    0.        33.4375     0.        41.4375    50.28125
   0.         0.        42.46875    0.        27.90625   36.21875 ]

겨울 6.8828
[ 3.60351562  4.15234375  2.8046875   3.47265625  3.24414062  3.18554688
  3.12109375  3.8359375   4.05859375  3.83203125  4.57421875  4.1484375
  4.65234375  4.39453125  4.5234375   3.74804688  2.64648438  3.5546875
  3.98828125  4.02734375  5.49609375  6.8828125   4.73046875  3.91210938
  0.          4.08203125  5.4296875 ]

오빠 1.5791
[ 0.52148438  0.73388672  0.75097656  0.57324219  0.42651367  0.28881836
  0.          0.25366211  0.          0.95605469  0.81396484  0.21325684
  0.18811035  0.          0.37670898  0.27514648  0.          0.51416016
  0.          0.22180176  0.          1.57910156  0.          0.          0.
  0.55029297  0.32177734]

하던데 0.52246
[ 0.     

## Reference

In [46]:
reference_model_name = '{}/subword_df_slot.pkl'.format(reference_model_folder)
subword_slot_ref, subword2index_ref = load_subword_slot(reference_model_name)
index2subword_ref = [word for word, _ in sorted(subword2index_ref.items(),key=lambda x:x[1])]

In [51]:
print_subword_df_distribution(subword_slot_ref, subword2index_ref, '입니다 팝니다 겨울 오빠 하던데 극장 엔진오일 상영 엔진 차량 악장 휴양지 터미네이터 구매 리스 영화 로봇 커피 미국 선수'.split())


입니다 14.844
[  6.25         0.           0.           0.           9.40625      0.           0.
   0.           0.           0.           0.           8.578125     0.
  14.84375     10.25        11.6796875    0.           0.           0.
   7.12109375   6.953125     0.           0.           0.           0.           0.
   0.           7.9453125    0.           0.           0.        ]

팝니다 0.23523
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.23522949  0.          0.          0.          0.
  0.08630371  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.05715942  0.18078613  0.          0.        ]

겨울 11.922
[  1.93457031   4.1640625    2.2421875    1.70117188   4.5          7.1171875
   6.6328125    2.97070312   5.53125      4.515625     3.79296875
   4.8203125    3.70507812   9.3046875    3.62890625   1.43554688
   5.91796875   2.5078125    4.